In [1]:
import pandas as pd
import csv
import numpy as np
import seaborn as sns
from tqdm import tqdm
import math
from scipy.stats import mannwhitneyu
import time
import os

In [2]:
######## DHS----------------
def isInRange(n, tbl):
    #https://stackoverflow.com/questions/9019581/what-does-numpy-apply-along-axis-perform-exactly
    #https://stackoverflow.com/questions/71377816/check-if-value-is-between-two-values-in-numpy-array
    # n: is sgRNA location in the tiling/phenotypic dataset
    # tbl: numpy file of dataframe of bigbed with chro, start, and end of peak filtered_peak_file.to_numpy()
    #apply_along_axis applies the supplied function along 1D slices of the input array, 
    #with the slices taken along the axis you specify. 
    return sum(np.apply_along_axis(lambda row: row[0] <= n <= row[1], 1, tbl)\
        .tolist())

def dhs_peak_overlap(actual_tiling_gene , dhs_chromo_txt, sgrna_location_col):
    # This function find whether guide overlaps with a peak in one chromosome 
    
    # actual_tiling: CRISPRi data (or any CRISPR data) with at least three columns 
    #                       [a location to indicate sgRNA, Gene name, Chromosome]
    #                       Note this DOES NOT require unique sgRNA location 
    # dhs_chromo_txt: ATAC seq from ENCODE in bigBed format and read through pyBigWig
    # chrom: string of chromosome such as 'chr1'
    # sgrna_location_col: string of column for sgRNA location
    
    # returns a dataframe, same as the actual_tilling but with an additional column overlap with peak to indicate 
    # whether theres an overlap between start and end of a peak and sgRNA location for one chromosome
    
    #----------------------------------------------------------------------------------------------------
        
    # select unique pam coord and chr and gene - remove duplicate 
    # tiling_lib: CRISPRi data (or any CRISPR data) with at least three columns 
    #                       [a location to indicate sgRNA, Gene name, Chromosome]
    #                       Note this should has unique sgRNA location 
    tiling_lib = actual_tiling_gene.copy()
    
    # change from string to int
    tiling_lib[sgrna_location_col] = list(map(int,tiling_lib[sgrna_location_col]))
    
    #obtain the smallest pam coord in a specific chromosome
    smallest_pam_coord = tiling_lib[sgrna_location_col].min()
    #obtain the largest pam coord in a specific chromosome
    largestest_pam_coord = tiling_lib[sgrna_location_col].max()
    
    #subset chrom number and having the end coord to be larger than the smallest pam coord
    
    # Retrieving bigBed entries in https://github.com/deeptools/pyBigWig explains
    # filtered_peak_file returns a list of tuple of (Start position in chromosome, End position in chromosome)
    # filtered_peak_file = dhs_chromo_txt.entries(chrom, smallest_pam_coord, largestest_pam_coord, withString=False) 
    filtered_peak_file = dhs_chromo_txt[(dhs_chromo_txt["start"] >= int(smallest_pam_coord)) & (dhs_chromo_txt["end"] <= int(largestest_pam_coord))].copy()
    
    # only kept unique peaks and make it into dataframe because of https://www.biostars.org/p/464618/
    # filtered_peak_file = pd.DataFrame(set(filtered_peak_file))
    filtered_peak_file = filtered_peak_file.drop_duplicates(subset = ["start", "end"])
    
    if len(filtered_peak_file) == 0:
        print('This gene does not has any peaks in DHS')
        tiling_lib.loc[:, 'overlap with peak'] = 0
    else:
    # iterating over every single sgRNA location in the tiling library/a dataset with phenotypic data
        peak_list = [isInRange(x, filtered_peak_file.to_numpy()) 
                     for x in np.nditer(tiling_lib[sgrna_location_col].to_numpy())]  

        #https://stackoverflow.com/questions/32573452/settingwithcopywarning-even-when-using-locrow-indexer-col-indexer-value
        tiling_lib.loc[:, 'overlap with peak'] = peak_list

    return tiling_lib
def get_gene_chr_combo(library_path):
    dir_list = os.listdir(library_path)
    file_split_remove_csv = [file.split('.')[0] for file in dir_list if file.endswith('.csv')]# remove files that doesnt end with csv
    files_combo = [file.split('_') for file in file_split_remove_csv]
    return(files_combo)

def dhs_all_genes(file_combo,library_gene_path, dhs_chrom_path, sgrna_location_col = 'sgRNA \'Cut\' Position'):
    library_dhs = pd.DataFrame()
    for comb in tqdm(file_combo ,desc='number of gene symbol'):
        print(comb)
        chrom_num = comb[0]
        gene = comb[1]
        chr_gene_library = pd.read_csv(f'{library_gene_path}{chrom_num}_{gene}.csv')
        chr_gene_library['chromosome'] = chrom_num
        chr_gene_library['Target Gene Symbol'] = gene
        chr_dhs = pd.read_csv(f'{dhs_chrom_path}{chrom_num}.csv')
        chr_dhs['chromosome'] = chrom_num
        gene_check = dhs_peak_overlap(chr_gene_library , chr_dhs, sgrna_location_col = sgrna_location_col)
        library_dhs = pd.concat([library_dhs,gene_check])
    return library_dhs

def standardize_chromosome(chromosome):
    chromosome = str(chromosome).replace('.0', '')
    if not chromosome.startswith('chr'):
        chromosome = 'chr' + chromosome
    return chromosome


In [3]:
inpath = '../../Data/CleanedExternalData/'
NewDatasetCombine = pd.read_csv(inpath + 'DatasetCombineCleaned_v3.csv', low_memory=False)
NewDatasetCombine['chromosome'] = NewDatasetCombine['chromosome'].apply(standardize_chromosome)

NewDatasetCombine['chromosome'] = NewDatasetCombine['chromosome'].replace('chr24','chrY')
NewDatasetCombine['chromosome'] = NewDatasetCombine['chromosome'].replace('chr23','chrX')

NewDatasetCombine['orginal phenotype'] = NewDatasetCombine['Phenotype scores-ave_Rep1_Rep2'].combine_first(NewDatasetCombine['z_scored_avg_lfc']).combine_first(NewDatasetCombine['rhoSignChangedAndAvged'])
NewDatasetCombine = NewDatasetCombine[['Target Gene Symbol', 'chromosome', 'sgRNA Context Sequence','sgRNA \'Cut\' Position',
       'condition', 'Domain', 'DataSet','orginal phenotype']]

In [4]:
processedCRISPRiOutpath = '../../Data/DHS/SectionedCRISPRi/'

CRISPRi_fileCombo = get_gene_chr_combo(processedCRISPRiOutpath)
len(CRISPRi_fileCombo)


658

In [5]:
dhs_out = '../../Data/DHS/SectionedCRISPRi/'
peakOverlapOut = '../../Data/DHS/PeakOverlap/'

In [6]:
CRISPRiFiles_DHS_HCT116 = dhs_all_genes(CRISPRi_fileCombo,processedCRISPRiOutpath, dhs_out+'HCT116/')

number of gene symbol:   1%|▏                   | 6/658 [00:00<00:11, 56.16it/s]

['chr14', 'HAUS4']
['chr19', 'OR10H3']
This gene does not has any peaks in DHS
['chr11', 'SF3B2']
['chr17', 'OR4D1']
This gene does not has any peaks in DHS
['chr7', 'MRPS24']
['chr20', 'DDX27']
['chr9', 'DCTN3']
['chr12', 'POP5']
['chr8', 'POLR2K']
['chrX', 'SAGE1']
This gene does not has any peaks in DHS
['chr19', 'MRPL34']


number of gene symbol:   3%|▌                  | 18/658 [00:00<00:13, 48.85it/s]

['chr11', 'RAD9A']
['chr2', 'MRPL19']
['chr1', 'FDPS']
['chr1', 'HEATR1']
['chr17', 'COX10']
['chr7', 'TRRAP']
['chr19', 'CYP2A13']
This gene does not has any peaks in DHS
['chr13', 'ARGLU1']
['chr13', 'TM9SF2']


number of gene symbol:   5%|▊                  | 30/658 [00:00<00:12, 51.74it/s]

...

['chr16', 'RPS15A']
['chr5', 'THG1L']
['chr6', 'CDC5L']
['chr1', 'GNL2']
['chr6', 'SNRNP48']
['chr19', 'UQCRFS1']
['chr12', 'MRPS35']
['chr19', 'FBL']
['chr18', 'AFG3L2']


number of gene symbol:  99%|█████████████████▉| 654/658 [00:14<00:00, 43.32it/s]

['chr11', 'CHEK1']
['chr5', 'LARS1']
['chr11', 'OR52A1']
This gene does not has any peaks in DHS
['chr17', 'MED1']
['chr7', 'NEUROD6']
This gene does not has any peaks in DHS
['chr13', 'RNF113B']
This gene does not has any peaks in DHS
['chr2', 'CARF']
['chr9', 'SMC2']
['chr11', 'MRPL17']
['chr17', 'ZNF830']


number of gene symbol: 100%|██████████████████| 658/658 [00:14<00:00, 46.44it/s]

['chr22', 'BCR']
['chr17', 'NLE1']
['chr11', 'OR4C11']
This gene does not has any peaks in DHS


In [7]:
CRISPRiFiles_DHS_HCT116['overlap with peak'].value_counts()

overlap with peak
0    140846
1     25778
Name: count, dtype: int64

In [8]:
CRISPRiFiles_DHS_HCT116.to_csv(f'{peakOverlapOut}HCT116.csv')

In [9]:
CRISPRiFiles_DHS_A549 = dhs_all_genes(CRISPRi_fileCombo,processedCRISPRiOutpath, dhs_out+'A549/')

number of gene symbol:   1%|▏                   | 6/658 [00:00<00:13, 46.67it/s]

['chr14', 'HAUS4']
['chr19', 'OR10H3']
This gene does not has any peaks in DHS
['chr11', 'SF3B2']
['chr17', 'OR4D1']
This gene does not has any peaks in DHS
['chr7', 'MRPS24']
['chr20', 'DDX27']
['chr9', 'DCTN3']
['chr12', 'POP5']
['chr8', 'POLR2K']


number of gene symbol:   2%|▎                  | 11/658 [00:00<00:17, 36.98it/s]

['chrX', 'SAGE1']
['chr19', 'MRPL34']
['chr11', 'RAD9A']
['chr2', 'MRPL19']
['chr1', 'FDPS']
['chr1', 'HEATR1']


number of gene symbol:   3%|▌                  | 19/658 [00:00<00:17, 35.62it/s]

...

['chr19', 'UQCRFS1']
['chr12', 'MRPS35']
['chr19', 'FBL']
['chr18', 'AFG3L2']
['chr11', 'CHEK1']
['chr5', 'LARS1']
['chr11', 'OR52A1']
This gene does not has any peaks in DHS
['chr17', 'MED1']
['chr7', 'NEUROD6']


number of gene symbol:  99%|█████████████████▉| 654/658 [00:17<00:00, 39.67it/s]

This gene does not has any peaks in DHS
['chr13', 'RNF113B']
This gene does not has any peaks in DHS
['chr2', 'CARF']
['chr9', 'SMC2']
['chr11', 'MRPL17']
['chr17', 'ZNF830']
['chr22', 'BCR']


number of gene symbol: 100%|██████████████████| 658/658 [00:17<00:00, 37.07it/s]

['chr17', 'NLE1']
['chr11', 'OR4C11']
This gene does not has any peaks in DHS


In [10]:
CRISPRiFiles_DHS_A549['overlap with peak'].value_counts()

overlap with peak
0    133532
1     33092
Name: count, dtype: int64

In [11]:
CRISPRiFiles_DHS_A549.to_csv(f'{peakOverlapOut}A549.csv')

In [12]:
CRISPRiFiles_DHS_K562 = dhs_all_genes(CRISPRi_fileCombo,processedCRISPRiOutpath, dhs_out+'K562/')

number of gene symbol:   1%|▏                   | 5/658 [00:00<00:13, 49.40it/s]

['chr14', 'HAUS4']
['chr19', 'OR10H3']
This gene does not has any peaks in DHS
['chr11', 'SF3B2']
['chr17', 'OR4D1']
This gene does not has any peaks in DHS
['chr7', 'MRPS24']
['chr20', 'DDX27']
['chr9', 'DCTN3']
['chr12', 'POP5']
['chr8', 'POLR2K']
['chrX', 'SAGE1']


number of gene symbol:   2%|▍                  | 15/658 [00:00<00:18, 34.78it/s]

['chr19', 'MRPL34']
['chr11', 'RAD9A']
['chr2', 'MRPL19']
['chr1', 'FDPS']
['chr1', 'HEATR1']
['chr17', 'COX10']
['chr7', 'TRRAP']


number of gene symbol:   3%|▋                  | 23/658 [00:00<00:19, 32.32it/s]

...

['chr6', 'SNRNP48']
['chr19', 'UQCRFS1']
['chr12', 'MRPS35']
['chr19', 'FBL']
['chr18', 'AFG3L2']
['chr11', 'CHEK1']
['chr5', 'LARS1']


number of gene symbol:  99%|█████████████████▊| 652/658 [00:18<00:00, 35.29it/s]

['chr11', 'OR52A1']
['chr17', 'MED1']
['chr7', 'NEUROD6']
This gene does not has any peaks in DHS
['chr13', 'RNF113B']
['chr2', 'CARF']
['chr9', 'SMC2']
['chr11', 'MRPL17']
['chr17', 'ZNF830']


number of gene symbol: 100%|██████████████████| 658/658 [00:18<00:00, 35.97it/s]

['chr22', 'BCR']
['chr17', 'NLE1']
['chr11', 'OR4C11']
This gene does not has any peaks in DHS


In [13]:
CRISPRiFiles_DHS_K562['overlap with peak'].value_counts()

overlap with peak
0    123680
1     42944
Name: count, dtype: int64

In [14]:
CRISPRiFiles_DHS_K562.to_csv(f'{peakOverlapOut}K562.csv')

In [16]:
DHS_K562_combined = pd.merge(NewDatasetCombine,CRISPRiFiles_DHS_K562, how = 'inner')

In [17]:
DHS_HCT116_combined = pd.merge(NewDatasetCombine,CRISPRiFiles_DHS_HCT116, how = 'inner')

In [18]:
DHS_A549_combined = pd.merge(NewDatasetCombine,CRISPRiFiles_DHS_A549, how = 'inner')

In [20]:
DHS_K562_combined['DHS'] = 'K562'
DHS_HCT116_combined['DHS'] = 'HCT116'
DHS_A549_combined['DHS'] = 'A549'

In [21]:
DHS_combined = pd.concat([DHS_K562_combined,DHS_HCT116_combined,DHS_A549_combined])


In [22]:
DHS_combined

,Target Gene Symbol,chromosome,sgRNA Context Sequence,sgRNA 'Cut' Position,condition,Domain,DataSet,orginal phenotype,overlap with peak,DHS
0,ACTR6,chr12,TGCTTCGAGCACATAGAGGGATCTGGGCCC,100198288.0,K562,Kox1,Nunez,-0.018881,0,K562
1,ACTR6,chr12,CTGCTTCGAGCACATAGAGGGATCTGGGCC,100198289.0,K562,Kox1,Nunez,0.009292,0,K562
2,ACTR6,chr12,GTCCACCTGCTTCGAGCACATAGAGGGATC,100198295.0,K562,Kox1,Nunez,0.000734,0,K562
3,ACTR6,chr12,GGTCCACCTGCTTCGAGCACATAGAGGGAT,100198296.0,K562,Kox1,Nunez,-0.008256,0,K562
4,ACTR6,chr12,GGGCTAGACTCTGTCACCTCCGAGGGGTCC,100198321.0,K562,Kox1,Nunez,0.001314,0,K562
...,...,...,...,...,...,...,...,...,...,...
431090,HMX1,chr4,GTGAGGGCGCGGGGAGGGTGGGTCAGGCGG,8871196.0,HCT116,Zim3,InHouse,-0.721298,0,A549
431091,SNUPN,chr15,GCGTGGGCGGGGCCTGGGGCGGGTCGGACT,75625191.0,HCT116,Zim3,InHouse,-1.293837,1,A549
431092,SNUPN,chr15,GGAGGGTGGAGCGTGGGCGGGGCCTGGGGC,75625201.0,HCT116,Zim3,InHouse,-1.423518,1,A549
431093,NKX2-2,chr20,CAACTCCCCCCTCCCCCGCCCGCCGGGGCA,21513745.0,HCT116,Zim3,InHouse,-1.040580,0,A549


In [23]:
DHS_combined['overlap with peak'].value_counts()

overlap with peak
0    1065460
1     227825
Name: count, dtype: int64

In [24]:
out_path = '../../Data/DHS/'
DHS_combined.to_csv(out_path+'DHS_PeakOverlap.csv', index = False)